**Upload File**

Upload a JSON file exported from GTM under Admin > Export Container

In [ ]:
from google.colab import files
uploaded = files.upload()

**Parse the Variables**

Drill down into the variable section of the JSON export. Extract the name and ID of each variable. Look for instances matching the name in double curly braces in the entire JSON export and set match to True to denote variables that are in use somewhere in the GTM container

Save the variable data to a dictionary and list the first five entries.

In [ ]:
%%capture
!pip install -U 'fsspec[s3]'

import json
import pandas as pd
import re

uploaded_json = list(uploaded.values())[0]

container = json.loads(uploaded_json.decode('utf-8'))

# drill down to the variables section of the container
variables = container['containerVersion']['variable'] 

rows = []

for var in variables:
  row = {}
  row['variableId'] = var['variableId']
  row['name'] = var['name']
  # variables can be found nearly anywhere; we need to look for the variable name in double curly braces {{var_name}}
  pattern = "{{" + row['name'] + "}}"
  if(re.search(pattern, json.dumps(container))):
    row['match'] = True
  else: 
    row['match'] = False

  rows.append(row)
rows[:5]

**Convert to DataFrame**

Extract the container ID to use as the name of the Google Sheet and then convert the dictionary to a DataFrame. 

In [ ]:
import re
container_id = re.findall( "GTM-[A-Z0-9]+" , str(uploaded.keys())) 

data = pd.DataFrame(rows)
data.head()

,variableId,name,match
0,1,url,True
1,2,referrer,True
2,3,event,False
3,4,URLParams,True
4,5,element url,True


**Authenticate**

This will open a new window where you need to log in to Google Accounts, give this Colab permission to access Drive, copy the key that gets generated after giving permission, and finally paste that key back here and complete the authentication.

*2022-04-19: Updated authentication method*

In [ ]:
# authenticate
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

**Save to Google Sheets**

The will save as a GSheet in your Google Drive with the title of "GTM Variables for GTM-CONTAINERID." It will take a few minutes for the data to fully populate in the sheet so be patient.

In [ ]:
# create, and save df
from gspread_dataframe import set_with_dataframe
title = 'GTM Variables for ' + container_id[0]
gc.create(title)  # if not exist
sheet = gc.open(title).sheet1
set_with_dataframe(sheet, data) 
# include_index=False, include_column_header=True, resize=False